In [ ]:
from graph_tool.all import * 
import numpy as np
import random as rdm
from itertools import islice
import matplotlib.pyplot as plt
from collections import defaultdict

## MISURE E METRICHE

In [ ]:
#Internal community cohesion
def Internal_community_cohesion(Graph, Local_Community, candidate_node=None):
    #Mi prendo i nodi della comunità locale (set) (Sorted perchè utilizzo lista)
    Loc_C = Local_Community.copy()
    #Adesso controllo che la mia local community non abbia come unico vicino il nodo candidato. Se è così gli attribuisco massima importanza. Esempio unico nodo connesso ad hub.
    neigh_loc = set()
    for node in Loc_C:
        # Prendo i vicini della community locale
        neigh_loc.update(Graph.vertex(node).out_neighbors())

    if candidate_node != None:
        if len(list(neigh_loc)) == 1:
            if list(neigh_loc)[0] == candidate_node:
                return 10
        Loc_C.append(candidate_node)
    #Inizializzo internal tightness
    ITC = 0
    lenght = len(Loc_C)
    Loc_C = list(Loc_C)
    #Per ogni core node e candidato nuovo vertice, calcolo la ITC
    for node in range(lenght):
        core_node = Loc_C[node]
        neighb = Graph.get_out_neighbors(core_node)
        neighbs = set(neighb).intersection(set(Loc_C))
        for negh in neighbs:
            if core_node < negh:
                ITC += ILW(Graph, core_node, negh)
                
    if ITC == 0 or ITC == None:
        return 0
    else:
        return ITC[0]



#External community cohesion
def External_community_cohesion(Graph, Local_Community, candidate_node=None):
    #Mi prendo i nodi della comunità locale (set) (Sorted perchè utilizzo lista)
    Loc_C = Local_Community.copy()
    if candidate_node != None:
        Loc_C.append(candidate_node)
    #aggiungo alla comunità anche il nodo candidato
    core_vertices_exp = sorted(Loc_C)
    #definisco cos'è un boundary node: Tutti i nodi nella community che hanno almeno un vicino in U (vedere paper)
    boundary_node = set()
    for node in core_vertices_exp:
        neigh = set()
        # Prendo i vicini della community locale
        neigh.update(Graph.vertex(node).out_neighbors())
        if neigh.difference(Loc_C):
            boundary_node.add(node)
    #Inizializzo external tightness
    ETC = 0
    #Per ogni boundary node e candidato nuovo vertice calcolo la ETC
    for bound_node in boundary_node:
        neighb = Graph.vertex(bound_node).out_neighbors()
        neigh_not_core_vertices_exp = set(neighb).difference(set(core_vertices_exp))
        for neighbb in neigh_not_core_vertices_exp:
            ETC += ILW(Graph, bound_node, neighbb)
    
    if ETC == 0 or ETC == None:
        return 1
    else:
        return ETC[0]


# Community Cohesion
def Community_Cohesion(Graph, Local_Community, candidate_node=None):
    ITC = Internal_community_cohesion(Graph, Local_Community, candidate_node)
    ETC = External_community_cohesion(Graph, Local_Community, candidate_node)
    if ETC == 0:
        return 0
    else:
        return ITC/ETC

In [ ]:
#Jaccard
def Jaccard(Graph,NodeU,NodeV):
    jaccard = vertex_similarity(Graph, vertex_pairs=[(NodeU, NodeV)])
    return jaccard

In [ ]:
def Jaccard_Temp(set1, set2):
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union if union != 0 else 0

In [ ]:
#inv-log-weight
def ILW(Graph,NodeU,NodeV):
    jaccard = vertex_similarity(Graph, vertex_pairs=[(NodeU, NodeV)], sim_type='inv-log-weight')
    return jaccard

In [ ]:
#Conductance
def compute_conductance(graph, community):
    cut_vertices = community
    # Calculate the conductance given a set of cut vertices
    boundary_edges = []
    total_weight = 0
    cut_weight = 0

    for v in cut_vertices:
        for e in graph.iter_all_edges(v):
            #Conto +1 per ogni link per calcolare Vol(S)
            total_weight += 1
            #Link che inizio o finisco in S, ma non entrambi
            if graph.vertex_index[e[0]] not in cut_vertices or graph.vertex_index[e[1]] not in cut_vertices:
                boundary_edges.append(e)
                cut_weight += 1

    den = min( total_weight , 2 *len(list(graph.edges()))- total_weight)
    if den > 0:
        conductance = cut_weight / den
    else:
        conductance = 0
    #print('cut_weight' , cut_weight, 'total_weight', total_weight, 'total_weight_compl', len(list(graph.edges()))- total_weight)
    return conductance

## Algoritmi

In [ ]:
#Search Friends
def take(n, iterable):
    """Return the first n items of the iterable as a list."""
    return islice(iterable, n)

def search_friends(g, LC, max_d): 
    ego_net = set()
    #creo nuova vp utile per definire successivamente il subgraph dell'ego_net
    vfiltered = g.new_vertex_property('bool')
    #creo un dizionario per i nodi utile a racchiuderne il degree interno e quello totale
    node_dict = dict()
    for node in LC:
        vert_prop, tuple = shortest_distance(g, node, max_dist=max_d, return_reached=True) 
        #aggiungo i nodi che trovo a distanza 1 per ogni songolo nodo nella local community
        ego_net = ego_net.union(set(tuple))
    #attribuisco la vertex property necessaria a creare il subgraph
    vfiltered.a[list(ego_net)] = True
    vfiltered.a[list(LC)] = True
    graph_filtered = GraphView(g, vfiltered) 
    #creo due vertex property in modo tale da calcolarmi il total degree dei nodi sia dentro il netwrok intero che nell'ego
    vpp_d_full = g.degree_property_map('total')
    vpp_d_ego = graph_filtered.degree_property_map('total')
    ego_not_in_LC = ego_net.difference(set(LC))
    for node in ego_not_in_LC:
        #Mi vado a calcolare il rapporto fra il degree interno all'ego e quello totale del network
        tot_degree = vpp_d_full[node]
        tot_degree_ego = vpp_d_ego[node]
        #definisco tale misura come friendship
        friendship = tot_degree_ego/tot_degree
        #Aggiungo il nodo con la friendship al dizionario dei nodi
        node_dict[node] = friendship
    #Ordino il dizionario in ordine decrescente in base alla friendship e poi vado a prendere i primi dieci elementi
    sorted_dict = sorted(node_dict.items(), key = lambda x: x[1], reverse=True)
    nodes = take(10, sorted_dict)

    
    return list(nodes)

In [ ]:
#Search Friends per espansione secondaria
def ExpandFriends(g, LC, max_d): 
    ego_net = set()
    #creo nuova vp utile per definire successivamente il subgraph dell'ego_net
    vfiltered = g.new_vertex_property('bool')
    vfiltered_ego_no_lc = g.new_vertex_property('bool')
    #creo un dizionario per i nodi utile a racchiuderne il degree interno e quello totale
    node_dict = dict()
    for node in LC:
        vert_prop, tuple = shortest_distance(g, node, max_dist=max_d, return_reached=True) 
        #aggiungo i nodi che trovo a distanza 1 per ogni songolo nodo nella local community
        ego_net = ego_net.union(set(tuple))
        
    #attribuisco la vertex property necessaria a creare il subgraph
    vfiltered.a[list(ego_net)] = True
    vfiltered.a[list(LC)] = True
    graph_filtered = GraphView(g, vfiltered) 
    #creo un subgraph con solo i nodi che fanno parte dell'ego ma non della LC per poi calcolarmi anche la lite_friendship. 
    ego_not_in_LC = ego_net.difference(set(LC))
    vfiltered_ego_no_lc.a[list(ego_not_in_LC)] = True
    graph_filtered_ego_no_lc = GraphView(g, vfiltered_ego_no_lc)
    #clacolo i vari degree totali
    vpp_d_full = g.degree_property_map('total')
    vpp_d_ego = graph_filtered.degree_property_map('total')
    vpp_d_ego_no_lc = graph_filtered_ego_no_lc.degree_property_map('total')
    for node in ego_not_in_LC:
        #Mi vado a calcolare il rapporto fra il degree interno all'ego e quello totale del network
        tot_degree = vpp_d_full[node]
        tot_degree_ego = vpp_d_ego[node]
        tot_degree_ego_no_lc = vpp_d_ego_no_lc[node] #questo valore mi dice il # di connessioni dei nodi nell'ego escludendo la LC.
        #definisco tale misura come friendship
        friendship = tot_degree_ego/tot_degree
        lite_friendship = tot_degree_ego_no_lc/tot_degree_ego
        #print(node, friendship, lite_friendship)
        #Aggiungo il nodo con la friendship al dizionario dei nodi
        node_dict[node] = [friendship, lite_friendship]
    
    return node_dict

In [ ]:
def CoreAreaDetection(Graph, seed_node, subgraphs = False, failure_node = None):
    if failure_node is None:
        failure_node = set()
    LC = set()
    LC.add(seed_node)
    #Inizializzo CT a 0
    Comm_cohesion = 0
    Comm_cohesion_pre = -1
    V = list(Graph.iter_vertices())
    while Comm_cohesion > Comm_cohesion_pre:
        #Al posto di cercare tutti i neighboors utilizzo funzione per cercare solo i top 10 per friendship
        neigh_comm = search_friends(Graph, LC, 1)
        if len(neigh_comm) == 0:
            vcommunity = Graph.new_vp("int32_t")
            for node in LC:
                vcommunity[Graph.vertex(node)] = 1
            return LC, 0, vcommunity, failure_node
        Candidates = []
        #per ogni top 10 (prendo [0] perchè la funzione mi ritorna una tuple con anche la misura di friendship) la Node CT che altro non è che il Delta nella CT causato dall'aggiunta di quel nodo alla LC
        #print('neigh_comm',neigh_comm[0])
        for node, _ in neigh_comm:
            #print('node', node)
            community_coe = Community_Cohesion(Graph, list(LC), node)
            difference_CT_NCT = community_coe-Comm_cohesion
            #print('difference_CT_NCT', difference_CT_NCT)
            #Se il Delta è positivo aggiungo il nodo ai candidati
            if difference_CT_NCT > 0:
                Candidates.append([node, difference_CT_NCT])
                #print('node', node, 'diff_CT', difference_CT_NCT, 'NCT', community_coe)
        #print('Candidates', Candidates)
        # Faccio distinzione dal fatto che ho solo il seed o anche altri nodi
        Candidates.sort(key=lambda x: x[1], reverse = True)
        for node, _ in Candidates:
            LC.add(int(node))
            #print('LC dopo add', LC)
            break
        #se non riesco a catturare nodi nella prima run della LC perchè ad esempio è ponte o è vertice esterno, allora aggiungo come tentivo il vicino che
        # ha la connessione più forte in termini di tot_d_ego/tot_d e metto a 0.0001 la CT
        while len(LC) == 1:
            if seed_node in failure_node:
                LC = LC.union(failure_node)
                break
            else:
                failure_node.add(seed_node)
                LC, Comm_cohesion, _, failure_node = CoreAreaDetection(Graph, int(neigh_comm[0][0]), failure_node = failure_node)
        LC = LC.union(failure_node)
        #print('best_candidates', best_candidates)
        # Da Vedere come calcolare per via del fatto che il candidate node ora qua non c'è.
        Comm_cohesion_pre = Comm_cohesion
        #print('pre_ct',Comm_cohesion)
        Comm_cohesion = Community_Tightness(Graph, LC)
        #print('post_ct',Comm_cohesion)
    
    vcommunity = Graph.new_vp("int32_t")
    for node in LC:
        vcommunity[Graph.vertex(node)] = 1
    if subgraphs == True:
        sub = GraphView(Graph, vfilt=vcommunity.a == 1)
        graph_draw(sub, vprop=Jaccard, vertex_text=sub.vertex_index)
    return LC, Comm_cohesion, vcommunity, failure_node

In [ ]:
def CoreAreaExtension(Graph, LC, p_lambda = 0.6, subgraphs = False):

    Conductance = compute_conductance(Graph, LC)
    Conductance_pre = 1
    LC_pre = {}
    #contatore per tenere conto se vi sono nodi aggiunti all'iniziale LC
    while Conductance_pre > Conductance:
        LC_pre = LC.copy()
        #Vado a ricreare un ego_network considerando l'intera area iniziale (LC)
        nodi_booster = search_friends_final(Graph, LC, 1)
        i = 0
        #print(i)
        for k,v in nodi_booster.items():
            #v[0] è tot_d_ego/tot_d mentre
            if v[0] > p_lambda and v[1] < v [0]: #la logica è che devo avere più connessioni nell'ego, ma queste devono essere maggiormente con la LC che crea l'ego
                LC.add(int(k))
                i +=1
        #print(LC)
        #Se ho aggiunto almeno un nodo vado a cercare per uno step altri eventuali nodi isolati
        if i > 0:
            new_nodi_booster = search_friends_final(Graph, LC, 1)
            for k,v in new_nodi_booster.items():
                #v[1] è tot_d_ego_no_lc/tot_d
                if (v[0] == 1 and v[1] == 0): #la logica qua implicata è che aggiungo i nodi che sono solo connessi con la community espansa e che sennò risulterebbero isolati
                    LC.add(int(k))
            Conductance_pre = Conductance
            Conductance = compute_conductance(Graph, LC)
        else:
            break
    
    
    Comm_cohesion = Community_Cohesion(Graph, LC_pre)

    vcommunity = Graph.new_vp("int32_t")
    for node in LC_pre:
        vcommunity[Graph.vertex(node)] = 1
    if subgraphs == True:
        sub = GraphView(Graph, vfilt=vcommunity.a == 1)
        graph_draw(sub, vprop=Jaccard, vertex_text=sub.vertex_index)
    
    return LC_pre, Comm_cohesion, vcommunity

In [ ]:
def LECD(Graph, seed, p_lambda = 0.6, subgraph_core_area = False, subgraph_LC = False):
    LC, _x, _y, _z = CoreAreaDetection(Graph, seed, subgraph_core_area)
    LC_finale, CT, vpp = CoreAreaExtension(Graph, LC, p_lambda, subgraph_LC)
    return LC_finale, CT, vpp

In [ ]:
def Partition_Extraction(Graph, seed):
    G = Graph
    #lista dei nodi all'interno del grafo
    V = list(G.iter_vertices())
    s = seed
    #set dei nodi già usati
    V_used = set()
    # dizionario delle comunità
    LC_Dict = {}
    #Creo nuova vertex property
    vcommunity = G.new_vp("int32_t")
    
    #Guardo al grado di tutti i nodi per andare ad escludere quelli isolati
    degree = G.get_total_degrees(V)
    V_no_isolated = set(v for v, de in zip(V, degree) if de != 0)

    #serve a nominare in via progressiva le comunitò
    i=0
    while len(V_no_isolated) != 0: 
        i+=1
        Local_comm, CH, vpp, _ = CoreAreaDetection(G, s)
        #vado a rimuovere i nodi che ho utilizzato nella comunità così che non li posso ripescare come seed
        V_no_isolated -= Local_comm
        #serve a vedere se un nodo già assegnato viene riasssegnato alla nuova comunità, se nessuno lo è si procede a crearne una nuova
        if not Local_comm.intersection(V_used):
            LC_Dict['LC_{}'.format(i)] = [Local_comm,  CH, compute_conductance(G, list(Local_comm))]
            for node in Local_comm:
                vcommunity[G.vertex(node)] = i
        else:
            # se qualche nodo già usato viene riutilizzato si va a calcolare con le altre comunità già esistenti la len(dell'intersezione)
            # e si utilizza come parametro per capire a quale comunità unire quella nuova.
            new_LC = Local_comm
            comm_to_delete = []
            for loc_comm, comm_stats in LC_Dict.items():
                score = len(comm_stats[0].intersection(new_LC))
                if score > 0:
                    new_LC = new_LC.union(comm_stats[0])
                    comm_to_delete.append(loc_comm)
                
            LC_Dict['LC_{}'.format(i)] = [new_LC, Community_Cohesion(G, new_LC), compute_conductance(G, list(new_LC))]
            for comm_to_del in comm_to_delete:
                del LC_Dict['{}'.format(comm_to_del)]
            for node in new_LC:
                vcommunity[G.vertex(node)] = i
        #si aggiorna il set dei nodi già usati
        V_used = V_used.union(Local_comm)
        if len(V_no_isolated) != 0:
            s = rdm.choice(list(V_no_isolated))
    return LC_Dict, vcommunity

In [ ]:
def Partition_Optimization(Graph, LC_Dict):
    LC_Dict_Copy = LC_Dict.copy()
    while len(LC_Dict_Copy) >1:
        #print('len_before', len(LC_Dict_Copy))
        Max_delta_CND = 0
        max_CND_LC = max(LC_Dict_Copy, key=lambda k: (LC_Dict_Copy[k][2], LC_Dict_Copy[k][1], LC_Dict_Copy[k]))
        if LC_Dict_Copy[max_CND_LC][2] > 0.30:
            Comm_under_evaluation = LC_Dict_Copy[max_CND_LC][0]
            #print('community_ue', Comm_under_evaluation, 'max_CND_LC', max_CND_LC)
            neigh_Comm_under_evaluation = set()
            for node in Comm_under_evaluation:
                neigh_Comm_under_evaluation.update(G.vertex(node).out_neighbors())
                # for neigh in G.iter_all_neighbors(node):
                    # neigh_Comm_under_evaluation.add(neigh)
            new_community = set()
            key_max =-100
            for key, values in LC_Dict_Copy.items():
                #print('comm_da comparare', values[0], 'LC', key)
                if key == max_CND_LC:
                    continue
                else:
                    if neigh_Comm_under_evaluation.intersection(values[0]):
                        new_comm = Comm_under_evaluation.union(values[0])
                        CH_new = Community_Cohesion(G, new_comm)
                        Cnd_Comm_new = compute_conductance(G, list(new_comm))
                        #print('CH_new', CH_new, 'Cnd_Comm_new', Cnd_Comm_new)
                        Delta_CH = CH_new - values[1]
                        Delta_CND = values[2] - Cnd_Comm_new
                        # print('Delta_CH', Delta_CH, 'Delta_CND', Delta_CND)
                        if Delta_CND >= Max_delta_CND and Delta_CH >= 0:
                            key_max = key
                            new_community = new_comm
                            #print('new_community', new_community)
                            Max_delta_CND = Delta_CND
                            #print('Max_delta_CND', Max_delta_CND)
            if new_community and new_community != Comm_under_evaluation:
                #print('pre_LC_DICT', LC_Dict)
                LC_Dict[key_max] = [new_community, Community_Cohesion(G, new_community), compute_conductance(G, list(new_community))]
                #print(new_community, Community_Cohesion(G, new_community), compute_conductance(G, list(new_community))
                del LC_Dict[max_CND_LC]
                #print('post_LC_DICT', LC_Dict)
                #print('pre_LC_DICT_copy', LC_Dict_Copy)
                LC_Dict_Copy[key_max] = [new_community, Community_Cohesion(G, new_community), compute_conductance(G, list(new_community))]
                del LC_Dict_Copy[max_CND_LC]
                #print('post_LC_DICT_copy', LC_Dict_Copy)
                for node in new_community:
                    vcommunity[G.vertex(node)] = key_max[3:]
            else:
                del LC_Dict_Copy[max_CND_LC]
        else:
            break
        #print('len_after', len(LC_Dict_Copy)) 
        #print('')
        #print('')
    return LC_Dict, vcommunity

In [ ]:
def GELSE(Graph, seed, opt=False):
    LC_D, vpp = Partition_Extraction(Graph, seed)
    if opt == True:
        LCs_Opt, vpp = Partition_Optimization(Graph, LC_D)
        return LCs_Opt
    return LC_D, vpp

In [ ]:
def TECD(list_g, seed, subgraphs = False, stats = False, evolution=False):
    graphs = list_g
    Dict_Temporal_communities = {}
    List_Subgraph = []
    list_vpp = []
    count = 1
    
    for T_graph in graphs:
        vcommunity = T_graph.new_vp("int32_t")
        #Local Community
        LC, _, _ = LECD(T_graph, seed)
        for node in LC:
            vcommunity[T_graph.vertex(node)] = 1
            list_vpp.append(vcommunity)
        #Aggiungo lC al dizionario
        Dict_Temporal_communities['tc_{}'.format(count)] = LC
        count += 1

        
        if subgraphs == True:
            sub = GraphView(T_graph, vfilt=vcommunity.a == 1)
            List_Subgraph.append(sub)
            
    if evolution:
        for i, comm in enumerate(List_Subgraph):
            if i == 0:
                vcolor = List_Subgraph[i].new_vp("string")
                nodes_removed = set(List_Subgraph[i].vertices()).difference(set(List_Subgraph[i+1].vertices()))
                for node in List_Subgraph[i].vertices():
                    if node in nodes_removed:
                        vcolor[node] = 'red'
                    else:
                        vcolor[node] = 'yellow'
                graph_draw(List_Subgraph[i], vprop=Jaccard, vertex_text=subgraph[i].vertex_index, vertex_fill_color=vcolor, output="SubGraph_T{}.png".format(i))

                
            else:
                nodes_added = set(list(Dict_Temporal_communities.values())[i]).difference(set(list(Dict_Temporal_communities.values())[i-1]))
                if len(List_Subgraph) != i+1:
                    nodes_removed = set(list(Dict_Temporal_communities.values())[i]).difference(set(list(Dict_Temporal_communities.values())[i+1]))
                vcolor = List_Subgraph[i].new_vp("string")
                for node in List_Subgraph[i].vertices():
                    if nodes_removed:
                        if node in nodes_added.intersection(nodes_removed):
                            vcolor[node] = 'purple'
                        elif node in nodes_added:
                            vcolor[node] = 'green'
                        elif node in nodes_removed:
                            vcolor[node] = 'red'
                        else:
                            vcolor[node] = 'yellow'
                    else:
                        if node in nodes_added:
                            vcolor[node] = 'green'
                        else:
                            vcolor[node] = 'yellow'
                graph_draw(List_Subgraph[i], vprop=Jaccard, vertex_text=List_Subgraph[i].vertex_index, vertex_fill_color=vcolor, output="SubGraph_T{}.png".format(i))


    if stats == True:
        J = []
        LCs = []
        for k, v in Dict_Temporal_communities.items():
            LCs.append(v)
        i = 0
        while i < len(LCs) -1:
            Jac = jaccard_temp(LCs[i], LCs[i+1])
            print('Jaccard between tc_{} and tc_{} is {}'.format(i, i+1, Jac))
            print('Nodes that have left the community: {}'.format(set(LCs[i]).difference(set(LCs[i]).intersection(set(LCs[i+1])))))
            print('Nodes that have remained in the community: {}'.format(set(LCs[i]).intersection(set(LCs[i+1]))))
            print('Nodes that have joined the community: {}'.format(set(LCs[i+1]).difference(set(LCs[i]))))
            print('')
            i += 1
    if subgraphs == True:
        return Dict_Temporal_communities, list_vpp, List_Subgraph
    else:
        return Dict_Temporal_communities, list_vpp